# Solving Large-Scale Linear Programming Models


In [1]:
# load libraries
import numpy as np
import scipy.sparse as sp

import cplex as cp

In [2]:
def linear_programming(direction, A, senses, b, c, l, u):
    # create an empty optimization problem
    prob = cp.Cplex()

    # add decision variables to the problem including their coefficients in objective and ranges
    prob.variables.add(obj = c.tolist(), lb = l.tolist(), ub = u.tolist())

    # define problem type
    if direction == "maximize":
        prob.objective.set_sense(prob.objective.sense.maximize)
    else:
        prob.objective.set_sense(prob.objective.sense.minimize)

    # add constraints to the problem including their directions and right-hand side values
    prob.linear_constraints.add(senses = senses.tolist(), rhs = b.tolist())

    # add coefficients for each constraint
    row_indices, col_indices = A.nonzero()
    prob.linear_constraints.set_coefficients(zip(row_indices.tolist(),
                                                 col_indices.tolist(),
                                                 A.data.tolist()))

    # solve the problem
    prob.solve()

    # check the solution status
    print(prob.solution.get_status())
    print(prob.solution.status[prob.solution.get_status()])

    # get the solution
    x_star = prob.solution.get_values()
    obj_star = prob.solution.get_objective_value()

    return(x_star, obj_star)

## The product mix problem

To meet the demands of its customers, a company manufactures its products in its own factories (inside production) or buys them from other companies (outside production). Inside production is subject to some resource constraints: each product consumes a certain amount of each resource. In contrast, outside production is theoretically unlimited. The problem is to determine how much of each product should be produced inside the company and how much outside, while minimizing the overall production cost, meeting the demand, and not exceeding the resource constraints.

The statement of the problem must specify the set of products and the set of resources. For each product, we need to know the inside and outside production costs, and for each resource we need to know the available capacity of that resource. Finally, we need to know the consumption of resources by the different products.

The problem can be modeled as a linear programming problem as follows:
\begin{align*}
\mbox{minimize} \;\;& \sum\limits_{p = 1}^{P} c_{1p} x_{1p} + \sum\limits_{p = 1}^{P} c_{2p} x_{2p} \\
\mbox{subject to:} \;\;& x_{1p} + x_{2p} \geq d_{p} \;\;\;\; p = 1, 2, \dots, P \\
\;\;& \sum\limits_{p = 1}^{P} a_{rp}x_{1p} \leq b_{r} \;\;\;\; r = 1, 2, \dots, R \\
\;\;& x_{1p} \geq 0 \;\;\;\; p = 1, 2, \dots, P\\
\;\;& x_{2p} \geq 0 \;\;\;\; p = 1, 2, \dots, P.
\end{align*}
In this formulation, $x_{1p}$ and $x_{2p}$ show the production amounts of product $p$ for inside and outside production, respectively. $c_{1p}$ shows the unit production cost of product $p$ for inside production, and $c_{2p}$ is the unit acqusition cost of product $p$ for outside production. $d_{p}$ is the total demand for product $p$. $a_{rp}$ is the amount of resource $r$ we need to produce one unit of product $p$, whereas $b_{r}$ shows the total capacity for resource $r$.

Let us assume that our company sells pasta products. We produce and sell three different pasta types, namely, kluski, capellini, and fettuccine.

|            | Inside cost |Outside cost | Total demand |
|:-----------|------------:|------------:|-------------:|
| kluski     | 0.6         | 0.8         | 100          |
| capellini  | 0.8         | 0.9         | 200          |
| fettuccine | 0.3         | 0.4         | 300          |

|       | kluski | capellini | fettuccine | Total capacity | 
|:------|-------:|----------:|-----------:|---------------:|
| flour | 0.5    | 0.4       | 0.3        | 200            |
| eggs  | 0.2    | 0.4       | 0.6        | 400            |

In this scenario, we obtain the following linear programming problem:
\begin{align*}
\mbox{minimize} \;\;& 0.6 x_{11} + 0.8 x_{12} + 0.3 x_{13} + 0.8 x_{21} + 0.9 x_{22} + 0.4 x_{23} \\
\mbox{subject to:} \;\;& 1.0 x_{11} \phantom{+ 0.0 x_{12} + 0.0 x_{13}\:\:} + 1.0 x_{21} \phantom{+ 0.0 x_{22} + 0.0 x_{23}\:\:} \geq 100\\
\;\;& \phantom{0.0 x_{11} +\:} 1.0 x_{12} \phantom{+ 0.0 x_{13} + 0.0 x_{21}\:\:} + 1.0 x_{22} \phantom{+ 0.0 x_{23}\:\:} \geq 200\\
\;\;& \phantom{0.0 x_{11} + 0.0 x_{12} +\:} 1.0 x_{13} \phantom{+ 0.0 x_{21} + 0.0 x_{22}\:\:} + 1.0 x_{23} \geq 300\\
\;\;& 0.5 x_{11} + 0.4 x_{12} + 0.3 x_{13} \phantom{+ 0.0 x_{21} + 0.0 x_{22} + 0.0 x_{23}\:\:} \leq 200\\
\;\;& 0.2 x_{11} + 0.4 x_{12} + 0.6 x_{13} \phantom{+ 0.0 x_{21} + 0.0 x_{22} + 0.0 x_{23}\:\:} \leq 400\\
\;\;& x_{11} \geq 0 \\
\;\;& x_{12} \geq 0 \\
\;\;& x_{13} \geq 0 \\
\;\;& x_{21} \geq 0 \\
\;\;& x_{22} \geq 0 \\
\;\;& x_{23} \geq 0.
\end{align*}

In [3]:
def product_mix_problem(products_file, resources_file):
    products = np.loadtxt(products_file)
    resources = np.loadtxt(resources_file)

    P = products.shape[0]
    R = resources.shape[0]

    c = np.concatenate((products[:, 1], products[:, 2]))
    senses = np.concatenate((np.repeat("G", P), np.repeat("L", R)))
    b = np.concatenate((products[:, 3], resources[:, P + 1]))
    l = np.repeat(0, 2 * P)
    u = np.repeat(cp.infinity, 2 * P)

    aij = np.concatenate((np.repeat(1, 2 * P),
                          resources[:, 1:(P + 1)].flatten()))
    row = np.concatenate((np.repeat(range(P), 2),
                          np.repeat(range(R), P) + P))
    col = np.concatenate((np.array(range(2 * P)).reshape(2, P).T.flatten(),
                          np.tile(range(P), R)))
    A = sp.csr_matrix((aij, (row, col)), shape = (P + R, 2 * P))

    #import matplotlib.pyplot as plt
    #plt.figure(figsize = (5.0, 6.0))
    #plt.spy(A, marker = "o", markersize = 6)
    #plt.show()

    x_star, obj_star = linear_programming("minimize", A, senses, b, c, l, u)
    return(x_star, obj_star)

In [4]:
x_star, obj_star = product_mix_problem("products3.txt", "resources2.txt")
print(x_star)
print(obj_star)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.01 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            90.000000
1
optimal
[100.0, 150.0, 300.0, 0.0, 50.0, 0.0]
315.0


In [5]:
P = 500
R = 200

np.random.seed(220)
products = np.hstack((np.array(range(P)).reshape(P, 1) + 1, 
                      np.random.uniform(0.3, 0.8, (P, 1)),
                      np.random.uniform(0.5, 1.0, (P, 1)),
                      np.random.randint(100, 900, (P, 1))))
print(products.shape)
np.savetxt(fname = "products{}.txt".format(P), 
           X = products, fmt = "%d %f %f %d")

resources = np.hstack((np.array(range(R)).reshape(R, 1) + 1, 
                       np.random.uniform(0.2, 0.4, (R, P)),
                       np.random.randint(1500, 1900, (R, 1))))
print(resources.shape)
np.savetxt(fname = "resources{}.txt".format(R), 
           X = resources, fmt = "%d{} %d".format(" ".join(np.repeat(" %f", P))))

(500, 4)
(200, 502)


In [6]:
x_star, obj_star = product_mix_problem("products500.txt", "resources200.txt")
print(x_star)
print(obj_star)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 92 rows and 184 columns.
Reduced LP has 608 rows, 816 columns, and 82416 nonzeros.
Presolve time = 0.04 sec. (14.32 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =         26539.379159
Iteration:   188   Dual objective     =        121400.652104
Iteration:   365   Dual objective     =        174961.625592
1
optimal
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 190.64724100259977, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 346.8262900520285, 0.0, 0.0, 

In [7]:
P = 2500
R = 1000

np.random.seed(220)
products = np.hstack((np.array(range(P)).reshape(P, 1) + 1, 
                      np.random.uniform(0.3, 0.8, (P, 1)),
                      np.random.uniform(0.5, 1.0, (P, 1)),
                      np.random.randint(100, 900, (P, 1))))
print(products.shape)
np.savetxt(fname = "products{}.txt".format(P), 
           X = products, fmt = "%d %f %f %d")

resources = np.hstack((np.array(range(R)).reshape(R, 1) + 1, 
                       np.random.uniform(0.2, 0.4, (R, P)),
                       np.random.randint(15000, 19000, (R, 1))))
print(resources.shape)
np.savetxt(fname = "resources{}.txt".format(R), 
           X = resources, fmt = "%d{} %d".format(" ".join(np.repeat(" %f", P))))

(2500, 4)
(1000, 2502)


In [8]:
x_star, obj_star = product_mix_problem("products2500.txt", "resources1000.txt")
print(x_star)
print(obj_star)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 2 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 441 rows and 882 columns.
Reduced LP has 3059 rows, 4118 columns, and 2063118 nonzeros.
Presolve time = 1.88 sec. (350.08 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =        132865.578516
Iteration:   128   Dual objective     =        184041.823898
Iteration:   362   Dual objective     =        282412.339772
Iteration:   641   Dual objective     =        406651.393740
Iteration:   946   Dual objective     =        540350.127292
Iteration:  1614   Dual objective     =        761173.453559
Iteration:  2012   Dual objective     =        852389.303465
Iteration:  2390   Dual objective     =        902927.158561

Dual simplex solved model.

1
optimal
[0.0, 0.0, 0.0